# Econ5150: Advanced Computing

Zhentao Shi

## Speed

* Python is a high-level language. 
  * In most cases, vectorization speeds up computation.
* Multiple CPUs for parallel execution
  * Save time after optimizing the code for speed.


## Vectorization

* Mathematical equivalence $\neq$ computation equivalence

* Speed matter in
  * Big data
  * Simulations
  * Hyper parameter tuning


* For example, the preferred algorithm in Lin, Shi, Wang and Yan (2022) 
  * 8 hours on 24 cores = 192 core-hours

* Code optimization is essential for such problems.

* Optimizing code takes human time.

* Tradeoff between human time and computer time.

### Econometrics Example

In OLS regression, under heteroskedasticity
$
\sqrt{n}\left(\widehat{\beta}-\beta_{0}\right)\stackrel{d}{\to}N\left(0,E\left[x_{i}x_{i}'\right]^{-1}\mathrm{var}\left(x_{i}e_{i}\right)E\left[x_{i}x_{i}'\right]^{-1}\right)
$
where $\mathrm{var}\left(x_{i}e_{i}\right)$ can be estimated by 

$$
\underset{\mathrm{opt1}}{\frac{1}{n}\sum_{i=1}^{n}x_{i}x_{i}'\widehat{e}_{i}^{2}}=\underset{\mathrm{opt2,3}}{\frac{1}{n}X'DX}=\underset{\mathrm{opt 4}}{\frac{1}{n}\left(X'D^{1/2}\right)\left(D^{1/2}X\right)}
$$

where $D$ is a diagonal matrix of $\left(\widehat{e}_{1}^{2},\widehat{e}_{2,}^{2},\ldots,\widehat{e}_{n}^{2}\right)$.

At least 4 mathematically equivalent ways:

1. literally sum $\hat{e}_i^2 x_i x_i'$  over $i=1,\ldots,n$ one by one.
2. $X' \mathrm{diag}(\hat{e}^2) X$, with a dense central matrix.
3. $X' \mathrm{diag}(\hat{e}^2) X$, with a sparse central matrix.
4. $X' D$ and then out-product.


In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix 
import random
import datetime
import math
import statistics
import matplotlib.pyplot as plt

In [ ]:
def lpm(n):
    # estimation in a linear probability model

    # set the parameters
    b0 = np.array([-1, 1])

    # generate the data
    e = np.random.normal(size=n)
    X = np.hstack((np.ones((n, 1)), np.random.normal(size=(n, 1))))
    Y = (X @ b0 + e >= 0)

    # OLS estimation
    bhat = np.linalg.inv(X.T @ X) @ (X.T @ Y)

    # calculate the t-value
    bhat2 = bhat[1]  # parameter we want to test
    e_hat = Y - X @ bhat
    return X, e_hat


We run the 3 estimators for the same data, and compare the time.

In [ ]:
import time
import numpy as np
from scipy.sparse import diags

# an example of robust variance matrix.
# compare the implementation via matrix and vectorization.

# n = 5000; Rep = 10 # large matrix

n = 10; Rep = 5000 # small matrix

for opt in range(1, 5):
    pts0 = time.time()
    
    # initialize the matrix for computing the variance-covariance matrix
    XXe2 = np.zeros((2, 2))

    # loop over the replications and compute the variance-covariance matrix
    for iter in range(Rep):
        np.random.seed(iter)
        X, e_hat = lpm(n)
        
        # compute the variance-covariance matrix using element-by-element calculation
        if opt == 1:
            for i in range(n):
                XXe2 += e_hat[i]**2 * np.matmul( X[i,].T, X[i,] )
        
        # compute the variance-covariance matrix using matrix multiplication with dense matrices
        elif opt == 2:
            e_hat2_M = np.zeros((n, n))
            np.fill_diagonal(e_hat2_M, e_hat**2)
            XXe2 = X.T @ e_hat2_M @ X
        
        # compute the variance-covariance matrix using matrix multiplication with sparse matrices
        elif opt == 3:
            e_hat2_M = diags(e_hat**2, format='csr')
            # The format='csr' argument specifies that the resulting sparse matrix 
            # should be in Compressed Sparse Row (CSR)
            XXe2 = X.T @ e_hat2_M @ X
        
        elif opt == 4:
            e_hat2_M = np.zeros((n, n))
            # Xe = X * np.repeat(e_hat[:, np.newaxis], 2, axis=1)
            Xe = X * e_hat.reshape(-1,1).repeat(2, axis=1)
            XXe2 =  Xe.T @ Xe
        

    print("n =", n, ", Rep =", Rep, ", opt =", opt, ",time =", time.time() - pts0, "\n")


* Sparse matrix is useful for big matrix. 
* If the matrix is small, imposing sparsity does not help with speed.


## Loops

* Python is efficient at managing the memory.
  * Pre-defintion is useful in other languages such as R and Matlab

**Example**: see `loop.py` and `loop.r`

* Empirical coverage probability of a Poisson distribution
* Write a DIY `CI` for confidence interval

## Parallel Computing

Parallel computing becomes essential when the data size is beyond the storage of a single computer.
Here we explore the speed gain of parallel computing on a multicore machine.

Here we explore the speed gain of parallel computing on a multicore machine.

The package `multiprocessing` is the choice for parallel computing in Python.

Example: See the py script 
`data_example/parallel.py`.

I encounter dead loops when running `multiprocessing` in Jupyter notebook on Windows.

If we have two CPUs running simultaneously, in theory we can cut the time to a half of that on a single CPU. Is that what happening in practice?

## Cloud Computing

* Remote servers are more powerful than personal computers
* Instruments for intensive jobs

* Cloud storage
* Cloud computing

### Workflow 

* No fundamental difference lies between local and cloud
* Prepare in the cloud serve the data and code
* Open a shell for communication, run the code, and collect the results

* Command-line interface (CLI)

### Open Source

* Strong justification for open-source languages
* Installation with no limitations

### User Experience

* Cost and barrier of remote computing have reduced significantly
* Remote desktop best mimics the familiar operation system on a local computer
* Internet latency
* CLI flexible

* Remote Jupyter notebook work via a web browser as an interface
* Mouse and keyboard are still local
* Command is send from the browser to the remote computer
* Results are send back to the browser for display
* IDEs also have file management, to partially replace WinSCP




* `SCRP` 
  * resembles a workplace environment in a small company
  * always online (with VPN connection)
  * more powerful than the best local computer we can afford.
* `Amazon Web Service Cloud` or `阿里云`
  * commercial service tailored according to budget
  * from tiny demonstrative display to big enterprise business applications
  * need to be configured

### CUHK Econ

* Students have access to powerful remote computers
* Cluster's single-core performance may not as good as your laptop
* The advantage of cluster comes from parallel execution

1. Log in `scrp-login-2.econ.cuhk.edu.hk`;
2. Upload scripts and data to the server;
3. In a shell, run `python <file_name.py> result_file_name.txt`;
4. To run a command in the background, add "`&`" at the end of the above command.

**Example**
```
conda info --env

conda activate /opt/network/anaconda3/envs/pytorch-2.4
python parallel.py

python <parallel.py> py_print_out.txt & 
```

### Long jobs

* Keep jobs running on the background 
* Terminal can be freed for other task

* Disconnect with the server and the task is still running
* Otherwise, the task will be terminated immediately once we close window, disconnect with the server, or lose Internet or VPN connection.

### Prepare in Advance

* Test the input and output in small scale on  local computer or graphic cloud server
  * Work in CLI means no interaction with intermediate results
  * Don't debug in CLI
* Think in advance how to retrieve the results
* Export key results as data files (Rdata or csv...) or graph files (pdf, jpeg, png). 


## Reproducibility

* Configure environment
* Virtual environment

* Virtual machine
* [Docker](https://hub.docker.com/repository/docker/ztshi/econ5821)